# Analysis
* Applied to a stack of images

In [ ]:
AIR_IMAGE_PATH =  '../../data_store/20221023/f6_stack_air.tif'
im = io.imread(AIR_IMAGE_PATH)[:, 300:800, 300:600]
plt.figure(figsize=(6,6))
io.imshow(im[11])
print(im.shape)

x_size = im.shape[2]
y_size = im.shape[1]

ft_stack = fftshift(fft2(im), axes=[-1,-2])
ft_abs = np.abs(ft_stack)
ft_log = np.log10(ft_abs)



In [ ]:
plt.figure(figsize=(6,6))
plt.title('2D FFT')

x_freqs = fftshift(fftfreq(x_size, d = length_per_pixel)) 
y_freqs = fftshift(fftfreq(y_size, d = length_per_pixel) )
X,Y = np.meshgrid(x_freqs, y_freqs)

plt.pcolormesh(X,Y,ft_log[7], shading='auto')
plt.colorbar()
plt.ylabel('y Frequency (cycle per pixel)')
plt.xlabel('x Frequency (cycle per pixel)')


In [ ]:
plt.title('Plot of the FFT in the y direction at kx=0')
air_range = np.array([7, 8, 9, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 17])

def plot_line(idx):
    data = ft_log[idx,:, x_size//2]
    plt.plot(y_freqs, data, label=air_range[idx])

plot_line(0)
plot_line(4)
plot_line(7)

plt.xlabel('Frequencies (cycle per pixel)')
plt.ylabel('Magnitude (log scale)')
plt.legend(title='axial distance (mm) ')

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Plot of the FFT in the y direction at kx=0')

plt.plot( y_freqs, (np.moveaxis(np.abs(ft_stack[:,:, x_size//2]), 0, -1)), label=air_range)
plt.legend()
plt.xlabel('Frequencies (cycle per pixel)')
plt.ylabel('Magnitude')


In [ ]:
import scipy as sp

plt.figure(figsize=(15,5))

t = np.linspace(0, y_size, y_size)


period = 6.23 # in pixels
shift = 4 # pixels

wave = 0.5*sp.signal.square(t) + 0.5
slice = 10

# Scaling the data
def scale_data(data):
    data_mid = (np.max(data) + np.min(data))/2
    data_mid = np.average(data)
    data_range = (np.max(data) - np.min(data))
    data = (data-data_mid)/data_range + 0.5
    return data

data = scale_data(im[slice,:, 0].astype('float64'))
plt.plot(period* (t-shift) / (2*np.pi), wave, label='rectangular wave' )
plt.plot(data, label='actual data')

# plt.xlim(300,400)

plt.title(f'Spatial data, distance {air_range[slice]} mm')
plt.ylabel('Magnitude')
plt.xlabel('Vertical coordinate (pixels)')

plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(15,5))
plt.title(f'Frequency domain estimate of MTF (scaled appropriately)')
plt.plot(np.abs(fftshift(fft(wave))), label='ideal grating')
plt.plot(np.abs(fftshift(fft(data))), label=f'{air_range[slice]}')
plt.plot(np.abs(fftshift(fft(scale_data(im[7,:, 0].astype('float64'))))), label=f'{air_range[7]} mm')
plt.plot(np.abs(fftshift(fft(scale_data(im[0,:, 0].astype('float64'))))), label=f'{air_range[0]} mm')
plt.grid()
plt.legend()
plt.show()

In [ ]:
WATER_IMAGE_PATH =  '../data_store/20221023/f6_stack_water.tif'
im_water = io.imread(WATER_IMAGE_PATH)[:, 300:800, 300:600]
plt.figure(figsize=(6,6))
io.imshow(im_water[11])
water_range = np.array([9, 10, 11, 12, 13, 14, 15, 16, 16.5, 17, 17.5, 18, 18.5, 19, 20, 21])

In [ ]:
def get_contrast(image_stack):
    """Obtains an estimate of the Michelson contrast

    Args:
        image_stack (np.array): focal scanning stack of 2D images, assuming to be the grating

    Returns:
        contrast_avg, contrast_std: mean and std deviation of contrast estimates
    """
    N_images = len(image_stack)
    N_sections = 20
    N_lines = 30

    results = np.zeros((N_images, N_lines, N_sections))
    maxes = np.zeros((N_images, N_lines))

    image_stack = image_stack.astype('float64')
    maxes = np.max(image_stack[:,:,0::5], axis=1)
    mins = np.min(image_stack[:,:,0::5], axis=1)

    # Michelson contrast
    contrast = (maxes - mins)/ (maxes + mins)
    contrast_avg = np.average(contrast, axis=1)
    contrast_std = np.std(contrast, axis=1)

    return contrast_avg, contrast_std

# Calculates the contrast of a stack of images of a grating in air and in water
air_contrast, air_contrast_std = get_contrast(im)
water_contrast, water_contrast_std = get_contrast(im_water)

# Plots the curve
plt.figure(figsize=(10,5))
plt.scatter(air_range, air_contrast/np.max(air_contrast), label='air')
plt.errorbar(air_range, air_contrast/np.max(air_contrast), yerr=air_contrast_std, label='air')
plt.scatter(water_range, water_contrast/np.max(water_contrast), label='water')
plt.errorbar(water_range, water_contrast/np.max(water_contrast), yerr=water_contrast_std, label='water')

plt.title('Estimates of depth of field with distance')
plt.xlabel('Axial coordinate (mm)')
plt.ylabel('Contrast (fractional)')
plt.grid(which='both')

plt.legend()


In [ ]:
PATH =  '../OPT Shared files/2022-10-31/2022-10-31-f6 depth sweep wide range stack/MMStack_Pos0.ome.tif'

im = io.imread(PATH)[:, 300:800, 300:600]
# plt.figure(figsize=(6,6))
# io.imshow(im_water[11])
dist_range = np.arange(25, 2, -1)

# Calculates the contrast of a stack of images of a grating in air and in water
air_contrast, air_contrast_std = get_contrast(im)

PATH =  '../OPT Shared files/2022-10-31/2022-10-31-f22 depth sweep wide range stack/MMStack_Pos0.ome.tif'
im = io.imread(PATH)[:, 300:800, 300:600]
dist_range_2 = np.arange(25, -1, -1)

# Calculates the contrast of a stack of images of a grating in air and in water
f22_contrast, f22_contrast_std = get_contrast(im)

# Plots the curve
plt.figure(figsize=(10,5))
plt.scatter(dist_range, air_contrast/np.max(air_contrast), label='f6')
plt.errorbar(dist_range, air_contrast/np.max(air_contrast), yerr=air_contrast_std)
plt.scatter(dist_range_2, f22_contrast/np.max(f22_contrast), label='f22')
plt.errorbar(dist_range_2, f22_contrast/np.max(f22_contrast), yerr=f22_contrast_std)

plt.title('Estimates of depth of field with distance')
plt.xlabel('Axial coordinate (mm)')
plt.ylabel('Contrast (fractional)')
plt.grid(which='both')

plt.legend()